## analyse de la base
pas d’infos client sauf age et genre\
les ventes ont lieu dans un magasin, qui pour moi donne pays et région\
visiblement le revenu final est imputé de taxes\

Les taxes sont-elles liées au magasin ?\
> calcul du taux de taxes par ligne puis comparaison

- Table produit
- table categorie
- table sous-categorie

- table client
- table categorie d’age

- table magasin
- table pays
- table etat

- table vente
- table mois

In [82]:
import numpy as np
import pandas as pd

df = pd.read_csv('./sales2.csv')

# calcul du taux de taxes
tot = (df['Unit_Price']*df['Order_Quantity'])
tax = (tot-df['Revenue'])/tot
tax = tax.round(2)
# ajout dans la table principale
df['tax'] = tax

# comparaison des valeurs de taxes pour chaque magasin
taxcomp = df.loc[:,['Country', 'State', 'tax']]
taxcompcount = taxcomp.groupby(['Country', 'State']).count()
print(taxcompcount.head())

# vérification des valeurs uniques pour l’état de tasmanie
taxcomp = taxcomp[taxcomp['State'] == 'Tasmania']
print(pd.unique(taxcomp['tax']))
print(taxcomp.shape[0])

# >> conclusion : les taxes ne dépendent pas d’un magasin
# > C’est plus complexe puisque chaque article est suceptible d’être taxé différemment
# on va donct stocker le profit dans la table des ventes

                             tax
Country   State                 
Australia New South Wales  10412
          Queensland        5220
          South Australia   1564
          Tasmania           724
          Victoria          6016
[0.19 0.18 0.2  0.17 0.21 0.25 0.  ]
724


In [117]:
# Faire les tables pour les produits
# il faut faire trois tables
# Categories | Sous-Catégories | Produits

prod = df.loc[:,['Product', 'Product_Category', 'Sub_Category', 'tax']]
prod = prod.groupby(['Product_Category', 'Sub_Category', 'Product']).count()
print(prod)

uniqueprod = pd.unique(df['Product'])
print(uniqueprod.shape[0])

# > on voit que certains produits ont le même nom mais ne sont pas dans les mêmes catégories


                                                                tax
Product_Category Sub_Category      Product                         
Accessories      Bike Racks        Hitch Rack - 4-Bike          592
                 Bike Stands       All-Purpose Bike Stand       452
                                   Hitch Rack - 4-Bike            4
                 Bottles and Cages Mountain Bottle Cage        2444
                                   Road Bottle Cage            2640
...                                                             ...
Clothing         Socks             Women's Mountain Shorts, S     4
                 Vests             Classic Vest, L              318
                                   Classic Vest, M              320
                                   Classic Vest, S              324
                                   Touring-1000 Yellow, 50        2

[138 rows x 1 columns]
130


In [73]:
# chercher les produits en double

nuprod = prod.index.get_level_values(2)
nuprod = pd.DataFrame(nuprod)
nuprod['n'] = range(len(nuprod))
nuprod.groupby('Product').count().sort_values('n', ascending=False).head(10)

,n
Product,
AWC Logo Cap,2
Fender Set - Mountain,2
Water Bottle - 30 oz.,2
"Touring-1000 Yellow, 50",2
"Sport-100 Helmet, Black",2
Hitch Rack - 4-Bike,2
"Women's Mountain Shorts, S",2
Bike Wash - Dissolver,2
HL Mountain Tire,1


In [91]:
# vérifier que les sous-categories ne sont pas en double

# factorize renvoie une liste d’id correspondant à la place dans la table originale
# et une liste des valeurs dont l’index correpsond aux ids
ids, values = pd.factorize(prod.index.get_level_values(1))
# fonction pour vérifier que les ids sont dans l’ordre,
# et que donc il n’y a pas de duplucation de sous-categories
is_sorted = lambda a: np.all(a[:-1] <= a[1:])
print(len(values))
# > pas de duplication

17


In [101]:
# création table des categories

categ = df['Product_Category'].drop_duplicates().reset_index()
categ['id'] = range(1,4)
categ.drop('index', axis=1, inplace=True)
print(categ)

catdic = categ.to_dict('tight')['data']
catdic = dict(catdic)
print(catdic)

  Product_Category  id
0      Accessories   1
1         Clothing   2
2            Bikes   3
{'Accessories': 1, 'Clothing': 2, 'Bikes': 3}


In [116]:
# création table des sous-categories
def format_data(source:pd.DataFrame,col1:str,col2:str,repldic:dict):
    res = source.loc[:,[col1, col2]].drop_duplicates().reset_index()
    res.drop('index', axis=1, inplace=True)
    res['id'] = range(1, res.shape[0]+1)
    res[col1].replace(repldic, inplace=True)
    return res

def create_repldict(source:pd.DataFrame):
    res = source.drop(source.columns[0], axis=1)
    res = res.to_dict('tight')['data']
    res = dict(res)
    return res

# créer les données pour la table sous categories
subcat = format_data(df, 'Product_Category', 'Sub_Category', catdic)

subdic = create_repldict(subcat)
subcat

,Product_Category,Sub_Category,id
0,1,Bike Racks,1
1,1,Bike Stands,2
2,1,Bottles and Cages,3
3,2,Caps,4
4,1,Cleaners,5
5,1,Fenders,6
6,2,Gloves,7
7,1,Helmets,8
8,1,Hydration Packs,9
9,2,Jerseys,10


In [115]:
prod_data = format_data(df, 'Sub_Category', 'Product', subdic)
prod_data

,Sub_Category,Product,id
0,1,Hitch Rack - 4-Bike,1
1,2,Hitch Rack - 4-Bike,2
2,2,All-Purpose Bike Stand,3
3,3,Mountain Bottle Cage,4
4,3,Water Bottle - 30 oz.,5
...,...,...,...
133,16,"Touring-2000 Blue, 60",134
134,17,"Touring-1000 Yellow, 50",135
135,17,"Classic Vest, L",136
136,17,"Classic Vest, M",137
